In [1]:
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.tokenizers import Tokenizer, WordTokenizer, CharacterTokenizer
from allennlp.data.fields import LabelField, TextField, Field
from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, TokenCharactersIndexer

from overrides import overrides
import pandas as pd

In [2]:

@DatasetReader.register('tncrl')
class TNCRLDatasetReader(DatasetReader):
    
    def __init__(self,
                 token_indexers = None, tokenizer = None, lazy = False) -> None:
        super().__init__(lazy=lazy)
        
        self._tokenizer = tokenizer or CharacterTokenizer()
        self._token_indexers = token_indexers or {'tokens': TokenCharactersIndexer()}
        
    @overrides
    def text_to_instance(self, before: str, after: int, class_: str) -> Instance:
        fields: Dict[str, Field] = dict()
        tokens_before = self._tokenizer.tokenize(before)
        tokens_after = self._tokenizer.tokenize(after)
        fields['tokens_before'] = TextField(tokens_before, self._token_indexers)
        fields['tokens_after'] = TextField(tokens_after, self._token_indexers)
        fields['class'] = LabelField(class_)
        return Instance(fields)
    
    @overrides
    def _read(self, file_path):
        train = pd.read_csv('data/ru_train.csv')
        for i in range(len(train)):
            yield self.text_to_instance(str(train.loc[i]['before']), 
                                        str(train.loc[i]['after']), 
                                        train.loc[i]['class'])

In [5]:
reader = TNCRLDatasetReader(lazy=True)

In [7]:
it = reader.read('data/ru_train.csv')

In [13]:
inst = next(it.__iter__())

In [27]:
inst.fields['tokens_before'].index()

TypeError: index() missing 1 required positional argument: 'vocab'